In [1]:
import pandas as pd

In [4]:
df = pd.read_csv('train.csv')

In [5]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [6]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [9]:
df.shape

(18285, 5)

In [7]:
df = df.dropna()

In [11]:
X = df.drop('label', axis = 1)

In [12]:
y = df['label']

In [13]:
y

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 18285, dtype: int64

In [14]:
X.shape

(18285, 4)

In [15]:
y.shape

(18285,)

In [17]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [18]:
voc_size = 5000

#### One Hot Representation

In [19]:
messages = X.copy()

In [20]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [21]:
messages.reset_index(inplace=True)

In [24]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\KIIT0001\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [25]:
## Data Preprocessing
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    review = [stemmer.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [26]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [27]:
corpus[0]

'hous dem aid even see comey letter jason chaffetz tweet'

In [34]:
onehot_repr = [one_hot(words, voc_size) for words in corpus]
print(onehot_repr)

[[2426, 817, 4023, 4993, 3654, 4441, 4702, 3679, 2382, 3298], [72, 4519, 1693, 3532, 2235, 3826, 2190], [295, 2284, 620, 3548], [4399, 4175, 4124, 1940, 4999, 2059], [2209, 2235, 4298, 4037, 4602, 4644, 2235, 4968, 4279, 3290], [1903, 3463, 2214, 780, 4648, 4525, 2937, 1428, 3854, 306, 1936, 3652, 3803, 3909, 2190], [3682, 2749, 283, 3932, 3669, 699, 3292, 252, 195, 839, 2309], [2490, 3037, 1221, 3789, 2546, 3867, 4525, 1061, 195, 839, 2309], [4972, 1710, 3206, 513, 100, 3745, 3688, 544, 4525, 3208], [4382, 4505, 3824, 2581, 846, 1139, 473, 1581], [2119, 2913, 2635, 748, 2714, 1204, 2080, 1314, 2141, 1392, 713], [1940, 2843, 3654, 3745, 4525, 2546], [979, 4734, 1006, 2289, 4866, 3741, 4412, 1584, 3680], [1784, 2415, 1468, 4185, 3304, 896, 1683, 195, 839, 2309], [289, 2848, 3138, 629, 3533, 195, 839, 2309], [400, 4203, 3351, 3632, 1942, 2649, 4798, 2909, 3436, 1150], [1159, 4817, 4519], [454, 4448, 4700, 2148, 4525, 4905, 1911, 2190], [428, 2828, 1693, 708, 2832, 3092, 226, 3706, 4644],

In [32]:
corpus[0]

'hous dem aid even see comey letter jason chaffetz tweet'

In [33]:
onehot_repr[0]

[2426, 817, 4023, 4993, 3654, 4441, 4702, 3679, 2382, 3298]

#### Embedding Representation

In [36]:
sent_length = 20
embedded_docs = pad_sequences(onehot_repr, padding='pre', maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 3679 2382 3298]
 [   0    0    0 ... 2235 3826 2190]
 [   0    0    0 ... 2284  620 3548]
 ...
 [   0    0    0 ...  195  839 2309]
 [   0    0    0 ... 1657 3934 3982]
 [   0    0    0 ... 2807 2486 2367]]


In [37]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 2426,
        817, 4023, 4993, 3654, 4441, 4702, 3679, 2382, 3298])

In [38]:
## Creating Model
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features))
model.build(input_shape=(None, 20))
model.add(LSTM(100))
model.add(Dense(1, activation = 'sigmoid')) # because output is binary
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 20, 40)         │       200,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 100)            │        56,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           101 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 256,501 (1001.96 KB)

 Trainable params: 256,501 (1001.96 KB)

 Non-trainable params: 0 (0.00 B)

None


In [39]:
len(embedded_docs), y.shape

(18285, (18285,))

In [40]:
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [41]:
X_final.shape, y_final.shape

((18285, 20), (18285,))

In [44]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

#### Model Training

In [45]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs = 10, batch_size = 64)

Epoch 1/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 7s 20ms/step - accuracy: 0.7770 - loss: 0.4243 - val_accuracy: 0.9162 - val_loss: 0.1972
Epoch 2/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - accuracy: 0.9433 - loss: 0.1420 - val_accuracy: 0.9195 - val_loss: 0.1979
Epoch 3/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 5s 23ms/step - accuracy: 0.9670 - loss: 0.0889 - val_accuracy: 0.9170 - val_loss: 0.2337
Epoch 4/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 5s 23ms/step - accuracy: 0.9750 - loss: 0.0694 - val_accuracy: 0.9110 - val_loss: 0.2652
Epoch 5/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - accuracy: 0.9840 - loss: 0.0502 - val_accuracy: 0.9069 - val_loss: 0.2969
Epoch 6/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - accuracy: 0.9898 - loss: 0.0357 - val_accuracy: 0.9114 - val_loss: 0.3622
Epoch 7/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - accuracy: 0.9934 - loss: 0.0255 - val_accuracy: 0.9072 - val_loss: 0.4184
Epoch 8/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - accuracy: 0.9963 - loss: 0.0152 - val_accu

##### Adding Dropout (to improve the performance of the model)

In [46]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

C:\Users\KIIT0001\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


#### Performance Metrics and Accuracy

In [47]:
y_pred = model.predict(X_test)

189/189 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step


In [48]:
y_pred = np.where(y_pred > 0.5, 1, 0)

In [50]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[1465, 1954],
       [1665,  951]], dtype=int64)

In [51]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.40033140016570007

In [52]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.47      0.43      0.45      3419
           1       0.33      0.36      0.34      2616

    accuracy                           0.40      6035
   macro avg       0.40      0.40      0.40      6035
weighted avg       0.41      0.40      0.40      6035

